In [2]:
# === ÉTAPE 2.9: SAUVEGARDE DU MODÈLE ET DES ARTEFACTS ===
print("\n" + "=" * 60)
print("💾 SAUVEGARDE DU MODÈLE ET DES ARTEFACTS")
print("=" * 60)

# 1. Sauvegarder le meilleur modèle
model_filename = 'fraud_detection_model.pkl'
joblib.dump(best_model, model_filename)
print(f"1. ✅ Modèle sauvegardé: {model_filename}")

# 2. Sauvegarder l'encodeur
encoder_filename = 'onehot_encoder.pkl'
joblib.dump(encoder, encoder_filename)
print(f"2. ✅ Encodeur sauvegardé: {encoder_filename}")

# 3. Sauvegarder la liste des features
features_info = {
    'numerical_features': numerical_features,
    'binary_features': binary_features,
    'categorical_features': categorical_features,
    'all_features': list(X.columns),
    'feature_dtypes': {col: str(dtype) for col, dtype in X.dtypes.items()}
}

with open('features_info.json', 'w', encoding='utf-8') as f:
    json.dump(features_info, f, indent=2, ensure_ascii=False)
print(f"3. ✅ Informations des features sauvegardées: features_info.json")

# 4. Sauvegarder les métriques
metrics_info = {
    'best_model': best_model_name,
    'test_metrics': {
        'accuracy': float(accuracy_score(y_test, y_pred_best)),
        'precision': float(precision_score(y_test, y_pred_best, zero_division=0)),
        'recall': float(recall_score(y_test, y_pred_best, zero_division=0)),
        'f1_score': float(f1_score(y_test, y_pred_best, zero_division=0)),
        'roc_auc': float(roc_auc_score(y_test, y_pred_proba_best)) if hasattr(best_model, "predict_proba") else None,
        'average_precision': float(average_precision)
    },
    'confusion_matrix': confusion_matrix(y_test, y_pred_best).tolist(),
    'cost_analysis': {
        'true_positives': int(tp),
        'false_positives': int(fp),
        'false_negatives': int(fn),
        'cost_undetected_frauds': float(cost_undetected),
        'cost_false_alarms': float(cost_false_alarm),
        'total_cost': float(total_cost)
    },
    'training_info': {
        'train_samples': int(X_train.shape[0]),
        'test_samples': int(X_test.shape[0]),
        'fraud_rate_train': float(y_train.mean()),
        'fraud_rate_test': float(y_test.mean()),
        'training_date': datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    }
}

with open('model_metrics.json', 'w', encoding='utf-8') as f:
    json.dump(metrics_info, f, indent=2, ensure_ascii=False)
print(f"4. ✅ Métriques du modèle sauvegardées: model_metrics.json")

# 5. Créer un rapport de performance
print("\n5. 📄 Génération du rapport de performance...")

# Créer le rapport avec formatage correct
report_content = """{}
RAPPORT DE PERFORMANCE - SYSTÈME DE DÉTECTION DE FRAUDE
{}
Date de génération: {}
Modèle sélectionné: {}

📊 PERFORMANCE SUR LE TEST SET
{}
Accuracy: {:.4f}
Precision: {:.4f}
Recall: {:.4f}
F1-Score: {:.4f}
ROC-AUC: {}

📈 MATRICE DE CONFUSION
{}
               Prédit Non-Fraude   Prédit Fraude
Réel Non-Fraude     {:12d}     {:10d}
Réel Fraude         {:12d}     {:10d}

💰 ANALYSE COÛT/BÉNÉFICE
{}
• Fraudes détectées: {}
• Fausses alertes: {}
• Fraudes manquées: {}
• Coût total estimé: {:,.0f} DZD

🔍 TOP 5 FEATURES IMPORTANTES
{}
""".format(
    '=' * 60,
    '=' * 60,
    datetime.now().strftime("%d/%m/%Y %H:%M:%S"),
    best_model_name,
    '─' * 40,
    accuracy_score(y_test, y_pred_best),
    precision_score(y_test, y_pred_best, zero_division=0),
    recall_score(y_test, y_pred_best, zero_division=0),
    f1_score(y_test, y_pred_best, zero_division=0),
    f"{roc_auc_score(y_test, y_pred_proba_best):.4f}" if hasattr(best_model, "predict_proba") else 'N/A',
    '─' * 40,
    confusion_matrix(y_test, y_pred_best)[0, 0],
    confusion_matrix(y_test, y_pred_best)[0, 1],
    confusion_matrix(y_test, y_pred_best)[1, 0],
    confusion_matrix(y_test, y_pred_best)[1, 1],
    '─' * 40,
    tp, fp, fn, total_cost,
    '─' * 40
)

# Ajouter les features importantes si disponibles
if hasattr(best_model, 'feature_importances_'):
    report_content += "\n"
    for i, (feature, importance) in enumerate(importance_df.head(5).values, 1):
        report_content += f"{i}. {feature}: {importance:.4f}\n"

with open('performance_report.txt', 'w', encoding='utf-8') as f:
    f.write(report_content)
print(f"   ✅ Rapport de performance sauvegardé: performance_report.txt")


💾 SAUVEGARDE DU MODÈLE ET DES ARTEFACTS
1. ✅ Modèle sauvegardé: fraud_detection_model.pkl
2. ✅ Encodeur sauvegardé: onehot_encoder.pkl
3. ✅ Informations des features sauvegardées: features_info.json
4. ✅ Métriques du modèle sauvegardées: model_metrics.json

5. 📄 Génération du rapport de performance...
   ✅ Rapport de performance sauvegardé: performance_report.txt
